# OpenAI Agents SDK - 리서치 팀 오케스트레이션

이 튜토리얼은 OpenAI Agents SDK를 활용하여 전문화된 에이전트들이 협업하는 고급 멀티 에이전트 리서치 시스템을 구축하는 방법을 다룬다. 실제 SDK 패턴을 기반으로 한 종합적인 리서치 워크플로우를 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv pydantic

---

## 1. 리서치 팀 오케스트레이션

이 챕터에서는 전문화된 에이전트들이 협업하여 종합적인 리서치를 수행하는 고급 멀티 에이전트 시스템을 구축한다.

### 학습 내용

- 리서치 데이터 모델 설계 (Pydantic 기반)
- 전문화된 리서치 에이전트 생성 (기획, 웹, 학술, 산업, 종합)
- ResearchTeamManager 클래스를 통한 워크플로우 관리
- 병렬 리서치 실행과 결과 종합
- 트레이싱과 모니터링 통합

### 48.1 환경 설정

In [1]:
import os
import asyncio
import json
from datetime import datetime
from typing import Dict, List, Optional, Any
from dataclasses import dataclass

from dotenv import load_dotenv
from pydantic import BaseModel, Field

from agents import Agent, Runner, function_tool, WebSearchTool, trace, custom_span, gen_trace_id

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 리서치 데이터 모델 정의

리서치 워크플로우에서 사용할 구조화된 데이터 모델을 정의한다.

In [2]:
class ResearchItem(BaseModel):
    """
    개별 리서치 항목을 정의하는 모델이다.
    """
    query: str           # 검색 쿼리
    reason: str          # 검색 이유
    agent_type: str      # 에이전트 유형: "web", "academic", "industry"


class ResearchPlan(BaseModel):
    """
    리서치 계획을 정의하는 모델이다.
    """
    searches: List[ResearchItem]   # 검색 항목 리스트
    approach: str                   # 접근 방식
    expected_duration: str          # 예상 소요 시간


class ResearchSummary(BaseModel):
    """
    리서치 결과 요약 모델이다.
    """
    summary: str                    # 요약
    key_findings: List[str]         # 핵심 발견
    source_count: int               # 출처 수
    confidence_score: float         # 신뢰도 점수 (0.0 ~ 1.0)


class FinalReport(BaseModel):
    """
    최종 리서치 보고서 모델이다.
    """
    executive_summary: str          # 경영진 요약
    key_findings: List[str]         # 핵심 발견
    detailed_analysis: str          # 상세 분석
    recommendations: List[str]      # 권장 사항
    limitations: List[str]          # 제한 사항
    sources_consulted: int          # 참고 출처 수


print("리서치 데이터 모델이 정의되었다:")
print("  - ResearchItem: 개별 리서치 항목")
print("  - ResearchPlan: 리서치 계획")
print("  - ResearchSummary: 리서치 결과 요약")
print("  - FinalReport: 최종 보고서")

리서치 데이터 모델이 정의되었다:
  - ResearchItem: 개별 리서치 항목
  - ResearchPlan: 리서치 계획
  - ResearchSummary: 리서치 결과 요약
  - FinalReport: 최종 보고서


### 리서치 도구 정의

에이전트들이 사용할 리서치 도구를 정의한다.

In [3]:
@function_tool
def academic_search(query: str, max_results: int = 3) -> List[Dict[str, str]]:
    """
    학술 논문과 연구 출판물을 검색한다.
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수
    
    Returns:
        학술 자료 리스트
    """
    # 시연용 모의 데이터 - 실제로는 Google Scholar API 등 사용
    return [
        {
            "title": f"학술 연구: {query} - 연구 {i+1}",
            "authors": "Dr. Kim, Dr. Park, Dr. Lee",
            "journal": "Journal of Advanced Research",
            "year": "2024",
            "abstract": f"이 동료 심사 연구는 {query}에 대해 엄격한 방법론을 통해 분석하였다...",
            "doi": f"10.1000/example.{i+1}"
        }
        for i in range(max_results)
    ]


@function_tool
def industry_analysis(topic: str, industry: str = "technology") -> Dict[str, Any]:
    """
    산업 보고서와 시장 분석을 가져온다.
    
    Args:
        topic: 분석 주제
        industry: 산업 분야
    
    Returns:
        산업 분석 데이터
    """
    # 시연용 모의 데이터 - 실제로는 산업 분석 API 사용
    return {
        "market_size": "500억 달러",
        "growth_rate": "연간 15% 성장", 
        "key_players": ["삼성전자", "LG전자", "SK하이닉스"],
        "trends": [
            f"{topic} 관련 트렌드 1: AI 통합 가속화",
            f"{topic} 관련 트렌드 2: 클라우드 전환 확대"
        ],
        "challenges": [
            f"{topic}의 과제 1: 규제 환경 변화",
            f"{topic}의 과제 2: 인력 확보 어려움"
        ],
        "opportunities": [
            f"{topic}의 기회 1: 신규 시장 진출",
            f"{topic}의 기회 2: 파트너십 확대"
        ]
    }


print("리서치 도구가 정의되었다:")
print("  - academic_search: 학술 자료 검색")
print("  - industry_analysis: 산업 분석")

리서치 도구가 정의되었다:
  - academic_search: 학술 자료 검색
  - industry_analysis: 산업 분석


### 전문화된 리서치 에이전트 생성

각 리서치 영역에 특화된 에이전트를 생성한다.

In [4]:
# 리서치 기획 에이전트
planner_agent = Agent(
    name="Research Planner",
    instructions="""
    당신은 리서치 기획 전문가입니다. 종합적인 리서치 계획을 수립하세요.
    
    리서치 주제가 주어지면 다음을 포함한 구조화된 계획을 만드세요:
    - 다양한 리서치 유형(웹, 학술, 산업)에 대한 구체적인 검색 쿼리
    - 각 검색에 대한 명확한 이유
    - 예상 접근 방식과 일정
    
    가치 있는 인사이트를 얻을 수 있는 실행 가능하고 구체적인 검색 쿼리 작성에 집중하세요.
    """,
    output_type=ResearchPlan,
)

print(f"✓ {planner_agent.name} 생성됨")

✓ Research Planner 생성됨


In [5]:
# 웹 리서치 전문가 에이전트
web_research_agent = Agent(
    name="Web Research Specialist",
    instructions="""
    당신은 최신 정보에 집중하는 웹 리서치 전문가입니다.
    
    검색 쿼리가 주어지면:
    - 가장 최신의 관련 정보 검색
    - 최근 개발과 트렌드에 집중
    - 권위 있는 출처 우선
    - 핵심 인사이트와 실용적 발견 추출
    
    신뢰도 평가와 함께 종합적인 요약을 제공하세요.
    """,
    tools=[WebSearchTool()],
    output_type=ResearchSummary,
)

print(f"✓ {web_research_agent.name} 생성됨")

✓ Web Research Specialist 생성됨


In [6]:
# 학술 리서치 전문가 에이전트
academic_research_agent = Agent(
    name="Academic Research Specialist",
    instructions="""
    당신은 학술 자료에 집중하는 학술 리서치 전문가입니다.
    
    리서치를 수행할 때:
    - 동료 심사 학술 논문 검색
    - 방법론과 과학적 엄격성에 집중
    - 검증된 발견과 통계 추출
    - 연구의 제한점과 공백 기록
    
    신뢰도 등급과 함께 증거 기반 요약을 제공하세요.
    """,
    tools=[academic_search],
    output_type=ResearchSummary,
)

print(f"✓ {academic_research_agent.name} 생성됨")

✓ Academic Research Specialist 생성됨


In [7]:
# 산업 리서치 전문가 에이전트
industry_research_agent = Agent(
    name="Industry Research Specialist", 
    instructions="""
    당신은 시장 인텔리전스에 집중하는 산업 리서치 전문가입니다.
    
    주제를 분석할 때:
    - 시장 데이터와 비즈니스 인텔리전스 수집
    - 주요 플레이어와 경쟁 환경 식별
    - 비즈니스 기회와 과제 평가
    - 전략적 인사이트 제공
    
    실행 가능한 비즈니스 인텔리전스에 집중하세요.
    """,
    tools=[industry_analysis],
    output_type=ResearchSummary,
)

print(f"✓ {industry_research_agent.name} 생성됨")

✓ Industry Research Specialist 생성됨


In [8]:
# 리서치 종합 에이전트
synthesis_agent = Agent(
    name="Research Synthesizer",
    instructions="""
    당신은 종합적인 보고서를 작성하는 리서치 종합 전문가입니다.
    
    여러 리서치 요약이 주어지면:
    - 모든 출처의 발견을 종합
    - 패턴과 핵심 인사이트 식별
    - 구조화된 종합 보고서 작성
    - 실행 가능한 권장 사항 제공
    - 제한점과 추가 연구 필요 영역 기록
    
    종합적이면서도 실행 가능한 경영진 수준의 보고서를 작성하세요.
    """,
    output_type=FinalReport,
)

print(f"✓ {synthesis_agent.name} 생성됨")
print("\n전문화된 리서치 에이전트 5개가 모두 생성되었다.")

✓ Research Synthesizer 생성됨

전문화된 리서치 에이전트 5개가 모두 생성되었다.


### 리서치 팀 매니저 클래스

리서치 팀 워크플로우를 오케스트레이션하는 매니저 클래스를 구현한다.

In [9]:
class ResearchTeamManager:
    """
    리서치 팀 워크플로우를 오케스트레이션하는 매니저 클래스이다.
    financial_research_agent/manager.py 패턴을 기반으로 한다.
    """
    
    def __init__(self):
        self.trace_id = gen_trace_id()
    
    async def run(self, research_topic: str, target_audience: str = "general") -> FinalReport:
        """
        전체 리서치 워크플로우를 실행한다.
        
        Args:
            research_topic: 리서치 주제
            target_audience: 대상 청중
        
        Returns:
            최종 보고서
        """
        
        with trace("Research Team Workflow", trace_id=self.trace_id):
            print(f"🔬 리서치 시작: {research_topic}")
            print(f"👥 대상 청중: {target_audience}")
            print(f"📊 Trace ID: {self.trace_id}")
            print("-" * 60)
            
            # Phase 1: 기획
            research_plan = await self._create_research_plan(research_topic, target_audience)
            
            # Phase 2: 병렬 리서치 실행
            research_results = await self._execute_research(research_plan)
            
            # Phase 3: 종합
            final_report = await self._synthesize_findings(research_topic, research_results)
            
            return final_report
    
    async def _create_research_plan(self, topic: str, audience: str) -> ResearchPlan:
        """
        종합적인 리서치 계획을 생성한다.
        """
        
        with custom_span("Planning Phase"):
            print("📋 Phase 1: 리서치 계획 수립 중...")
            
            planning_input = f"""
            리서치 주제: {topic}
            대상 청중: {audience}
            
            다음에 대한 구체적인 검색 쿼리를 포함한 종합적인 리서치 계획을 수립하세요:
            1. 웹 리서치 (현재 트렌드, 뉴스, 개발 동향)
            2. 학술 리서치 (학술 논문, 연구)
            3. 산업 리서치 (시장 데이터, 비즈니스 인텔리전스)
            
            각 검색 쿼리를 구체적이고 집중적으로 만드세요.
            """
            
            result = await Runner.run(planner_agent, planning_input)
            plan = result.final_output_as(ResearchPlan)
            
            print(f"✅ {len(plan.searches)}개 검색이 포함된 리서치 계획 완성")
            print(f"   접근 방식: {plan.approach}")
            return plan
    
    async def _execute_research(self, plan: ResearchPlan) -> List[ResearchSummary]:
        """
        리서치 검색을 병렬로 실행한다.
        """
        
        with custom_span("Research Execution Phase"):
            print("🔍 Phase 2: 리서치 검색 실행 중...")
            
            # 병렬 실행을 위한 태스크 생성
            tasks = []
            for search_item in plan.searches:
                task = asyncio.create_task(self._perform_search(search_item))
                tasks.append(task)
            
            # 적절한 에러 처리와 함께 검색 실행
            results = []
            for task in asyncio.as_completed(tasks):
                try:
                    result = await task
                    if result is not None:
                        results.append(result)
                except Exception as e:
                    print(f"   ⚠️ 검색 실패: {e}")
                    continue
            
            print(f"✅ {len(results)}개 리서치 검색 완료")
            return results
    
    async def _perform_search(self, search_item: ResearchItem) -> Optional[ResearchSummary]:
        """
        적절한 에이전트로 단일 리서치 검색을 수행한다.
        """
        
        search_input = f"""
        검색 쿼리: {search_item.query}
        리서치 목적: {search_item.reason}
        
        철저한 리서치를 수행하고 종합적인 요약을 제공하세요.
        """
        
        try:
            # 유형에 따라 적절한 에이전트로 라우팅
            if search_item.agent_type == "web":
                agent = web_research_agent
            elif search_item.agent_type == "academic":
                agent = academic_research_agent
            elif search_item.agent_type == "industry":
                agent = industry_research_agent
            else:
                agent = web_research_agent  # 기본값
            
            print(f"   → {agent.name}: {search_item.query[:50]}...")
            result = await Runner.run(agent, search_input)
            return result.final_output_as(ResearchSummary)
            
        except Exception as e:
            print(f"   ❌ '{search_item.query}' 검색 실패: {e}")
            return None
    
    async def _synthesize_findings(self, topic: str, research_results: List[ResearchSummary]) -> FinalReport:
        """
        모든 리서치 발견을 최종 보고서로 종합한다.
        """
        
        with custom_span("Synthesis Phase"):
            print("📊 Phase 3: 발견 종합 중...")
            
            # 종합 입력 준비
            findings_summary = []
            total_sources = 0
            
            for i, result in enumerate(research_results, 1):
                findings_summary.append(f"""
                리서치 스트림 {i}:
                요약: {result.summary}
                핵심 발견: {', '.join(result.key_findings)}
                출처 수: {result.source_count}
                신뢰도: {result.confidence_score}
                """)
                total_sources += result.source_count
            
            synthesis_input = f"""
            리서치 주제: {topic}
            총 리서치 스트림: {len(research_results)}
            참고한 총 출처 수: {total_sources}
            
            리서치 발견:
            {''.join(findings_summary)}
            
            모든 발견을 종합하는 포괄적인 최종 보고서를 작성하세요.
            경영진 요약, 핵심 인사이트, 상세 분석, 실행 가능한 권장 사항을 포함하세요.
            """
            
            result = await Runner.run(synthesis_agent, synthesis_input)
            final_report = result.final_output_as(FinalReport)
            
            print(f"✅ 최종 보고서 생성 완료")
            print(f"   참고 출처: {final_report.sources_consulted}개")
            print(f"   핵심 발견: {len(final_report.key_findings)}개")
            print(f"   권장 사항: {len(final_report.recommendations)}개")
            
            return final_report


print("ResearchTeamManager 클래스가 정의되었다.")

ResearchTeamManager 클래스가 정의되었다.


### 리서치 팀 실행 데모

In [10]:
async def demonstrate_research_team():
    """
    리서치 팀의 실제 동작을 시연한다.
    """
    
    print("🚀 리서치 팀 오케스트레이션 데모")
    print("=" * 60)
    print("이 데모는 실제 SDK 패턴을 사용하는 멀티 에이전트 리서치 팀을 보여준다.")
    print()
    
    # 리서치 매니저 생성
    manager = ResearchTeamManager()
    
    # 리서치 워크플로우 실행
    result = await manager.run(
        research_topic="인공지능이 의료 서비스 제공과 환자 결과에 미치는 영향",
        target_audience="의료 경영진"
    )
    
    # 결과 표시
    print("\n" + "🎯 리서치 완료" + "\n" + "=" * 60)
    
    print(f"\n📝 경영진 요약:")
    print(f"  {result.executive_summary[:300]}...")
    
    print(f"\n🔑 핵심 발견 ({len(result.key_findings)}개):")
    for i, finding in enumerate(result.key_findings[:5], 1):
        print(f"  {i}. {finding}")
    if len(result.key_findings) > 5:
        print(f"  ... 외 {len(result.key_findings) - 5}개")
    
    print(f"\n💡 권장 사항 ({len(result.recommendations)}개):")
    for i, rec in enumerate(result.recommendations[:5], 1):
        print(f"  {i}. {rec}")
    if len(result.recommendations) > 5:
        print(f"  ... 외 {len(result.recommendations) - 5}개")
    
    print(f"\n📚 참고 출처: {result.sources_consulted}개")
    
    if result.limitations:
        print(f"\n⚠️ 제한 사항:")
        for limitation in result.limitations[:3]:
            print(f"  - {limitation}")
    
    print(f"\n📄 상세 분석 미리보기:")
    print(f"  {result.detailed_analysis[:200]}...")
    
    return result

# 데모 실행
result = await demonstrate_research_team()

🚀 리서치 팀 오케스트레이션 데모
이 데모는 실제 SDK 패턴을 사용하는 멀티 에이전트 리서치 팀을 보여준다.

🔬 리서치 시작: 인공지능이 의료 서비스 제공과 환자 결과에 미치는 영향
👥 대상 청중: 의료 경영진
📊 Trace ID: trace_0d317e133f094b0d8c8147bf2a8eb88d
------------------------------------------------------------
📋 Phase 1: 리서치 계획 수립 중...
✅ 9개 검색이 포함된 리서치 계획 완성
   접근 방식: 웹, 학술, 산업 각 리서치별로 최신 정보와 신뢰도 높은 근거자료를 획득하기 위해 공식 리포트, PubMed/Google Scholar, KISTI, 시장조사업체(Statista, Frost & Sullivan 등)의 데이터를 병행 수집합니다. 각 쿼리별로 대표 사례 및 핵심지표 정리, 실무 적용성 중심 요약, 의료 경영진을 위한 전략적 인사이트 도출을 목표로 하여 체계적으로 자료를 정리합니다.
🔍 Phase 2: 리서치 검색 실행 중...
   → Web Research Specialist: AI in healthcare 2024 trends and latest news...
   → Web Research Specialist: Recent advancements in clinical decision support s...
   → Web Research Specialist: 인공지능 활용으로 향상된 국내외 의료 서비스 사례...
   → Web Research Specialist: "impact of artificial intelligence on patient outc...
   → Web Research Specialist: "Artificial intelligence enabled diagnostics and p...
   → Web Research Specialist: Effectiveness of AI-driven predictive a

### 커스텀 리서치 주제 실행

In [12]:
async def run_custom_research(topic: str, audience: str):
    """
    사용자 정의 리서치 주제로 팀을 실행한다.
    
    Args:
        topic: 리서치 주제
        audience: 대상 청중
    """
    print(f"\n{'='*60}")
    print(f"커스텀 리서치 실행")
    print(f"주제: {topic}")
    print(f"대상: {audience}")
    print("="*60)
    
    manager = ResearchTeamManager()
    report = await manager.run(topic, audience)
    
    print(f"\n🎯 리서치 완료")
    print(f"핵심 발견 수: {len(report.key_findings)}")
    print(f"권장 사항 수: {len(report.recommendations)}")
    print(f"참고 출처 수: {report.sources_consulted}")
    
    return report

# 예시: 다른 주제로 리서치 실행
custom_report = await run_custom_research(
    topic="전기차 배터리 기술의 최신 동향과 미래 전망",
    audience="자동차 산업 투자자"
)


커스텀 리서치 실행
주제: 전기차 배터리 기술의 최신 동향과 미래 전망
대상: 자동차 산업 투자자
🔬 리서치 시작: 전기차 배터리 기술의 최신 동향과 미래 전망
👥 대상 청중: 자동차 산업 투자자
📊 Trace ID: trace_3439a0ca4f6e443ea60b101207b04f96
------------------------------------------------------------
📋 Phase 1: 리서치 계획 수립 중...
✅ 9개 검색이 포함된 리서치 계획 완성
   접근 방식: 웹, 학술, 산업 리서치 간 균형 있게 정보를 수집합니다. 1단계로 웹 리서치로 시장의 전반적 트렌드와 최신 이슈를 신속 포착한 뒤, 2단계에서 학술 논문과 정밀 연구로 기술의 핵심 성능 및 한계, 미래 가능성을 확인합니다. 마지막으로 산업 리서치로 시장 규모, 투자 기회, 주요 기업 전략, 공급망 구조 등 투자 판단에 필요한 실질적 정보와 예측 자료를 통합 검토합니다.

각 검색 쿼리는 대표적 데이터베이스, 영향력 있는 미디어, 업계 전문 리포트 등 신뢰성 높은 자료원 중심으로 설계했습니다.
🔍 Phase 2: 리서치 검색 실행 중...
   → Web Research Specialist: "2024 electric vehicle battery technology trends" ...
   → Web Research Specialist: "solid-state battery breakthroughs" site:news OR s...
   → Web Research Specialist: "lithium iron phosphate vs. nickel manganese cobal...
   → Web Research Specialist: "next-generation electric vehicle batteries" AND (...
   → Web Research Specialist: "battery degradation and lifecycle in EVs" AND 

---

## 리서치 팀 아키텍처 요약


📊 데이터 모델:
   - ResearchItem: 개별 검색 쿼리와 메타데이터
   - ResearchPlan: 전체 리서치 전략과 검색 목록
   - ResearchSummary: 개별 리서치 결과 요약
   - FinalReport: 종합된 최종 보고서

🤖 전문화된 에이전트:
   1. Research Planner: 리서치 계획 수립
   2. Web Research Specialist: 웹 기반 최신 정보 수집
   3. Academic Research Specialist: 학술 자료 분석
   4. Industry Research Specialist: 산업/시장 인텔리전스
   5. Research Synthesizer: 결과 종합 및 보고서 작성

🔄 워크플로우 단계:
   Phase 1: 기획 (Planning)
            - 주제 분석
            - 검색 전략 수립
            - 에이전트 할당 계획

   Phase 2: 실행 (Execution)
            - 병렬 리서치 수행
            - 에이전트별 전문 검색
            - 결과 수집

   Phase 3: 종합 (Synthesis)
            - 결과 통합
            - 패턴 식별
            - 최종 보고서 생성

🛠️ 핵심 기술:
   - Pydantic 구조화된 출력
   - asyncio 병렬 처리
   - trace/custom_span 모니터링
   - 에러 처리 및 복원력

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK를 활용한 리서치 팀 오케스트레이션을 다루었다. 다음과 같은 내용을 학습하였다:

1. **리서치 데이터 모델 설계**
   - ResearchItem, ResearchPlan, ResearchSummary, FinalReport 모델
   - Pydantic을 활용한 구조화된 데이터 관리

2. **전문화된 리서치 에이전트**
   - Research Planner: 전략 수립
   - Web/Academic/Industry Specialists: 영역별 전문 검색
   - Research Synthesizer: 결과 종합

3. **ResearchTeamManager 클래스**
   - 3단계 워크플로우 (기획 → 실행 → 종합)
   - 병렬 리서치 실행
   - 에러 처리 및 복원력

4. **모니터링 및 트레이싱**
   - trace, custom_span을 통한 워크플로우 추적
   - 실행 상태 모니터링

이러한 패턴을 활용하면 복잡한 리서치 작업을 자동화하고 고품질의 종합 보고서를 생성하는 멀티 에이전트 시스템을 구축할 수 있다.